In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns

import warnings; warnings.simplefilter('ignore')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


%matplotlib inline
import warnings; warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
import sklearn as sk
import mglearn
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from osgeo import gdal
from PIL import Image

In [2]:
def read_xyz(fileName):
    
    xyz_coordinates = [] #put xyz in an array
    
    with open(fileName,"r") as file:
        for line_number,line in enumerate(file):
            x,y,z = line.split()
            
            xyz_coordinates.append([int(x),int(y),float(z)])
    return xyz_coordinates

In [3]:
groundwater_map = read_xyz("GRW_MBS_50m.xyz")


[[490025, 6299975, 4.201082229614258],
 [490075, 6299975, 4.524005889892578],
 [490125, 6299975, 0.688279390335083],
 [490175, 6299975, 1.2161840200424194],
 [490225, 6299975, 0.06308009475469589],
 [490275, 6299975, -inf],
 [490325, 6299975, -inf],
 [490375, 6299975, -inf],
 [490425, 6299975, -inf],
 [490475, 6299975, -inf],
 [490525, 6299975, -inf],
 [490575, 6299975, -inf],
 [490625, 6299975, -inf],
 [490675, 6299975, -inf],
 [490725, 6299975, -inf],
 [490775, 6299975, -inf],
 [490825, 6299975, -inf],
 [490875, 6299975, -inf],
 [490925, 6299975, -inf],
 [490975, 6299975, -inf],
 [491025, 6299975, -inf],
 [491075, 6299975, -inf],
 [491125, 6299975, -inf],
 [491175, 6299975, -inf],
 [491225, 6299975, -inf],
 [491275, 6299975, -inf],
 [491325, 6299975, -inf],
 [491375, 6299975, -inf],
 [491425, 6299975, -inf],
 [491475, 6299975, -inf],
 [491525, 6299975, -inf],
 [491575, 6299975, -inf],
 [491625, 6299975, -inf],
 [491675, 6299975, -inf],
 [491725, 6299975, -inf],
 [491775, 6299975, -in

In [4]:
#display in dataframe 
#add cplumns of name
my_array = np.array(groundwater_map)

df = pd.DataFrame(my_array, columns = ['XKoordinat','YKoordinat','Depth'])



Now we have a table with x,y coordinates and depths. Next step is to copmare with csv file coordinates and get a column to state the ground water level.

In [5]:
data = pd.read_csv("Energi_Viborg_Dandas_data.csv")


In [6]:
#drop columns not needed after asking the company about the meaning of these features

columns_to_be_removed = ['ID', 'mslink','LedningID','Dobbeltled','EjerKompon','SystemKode','KategoriAf','DatoUdf']
data=data.drop(columns_to_be_removed,axis='columns')

In [7]:
# in the column DatoSaneri is the date of repairing and if there is no date it means it is not repaired

data['DatoSaneri'].fillna(0, inplace=True)

In [8]:
# take only the pipes that are broken(by TV insection) now and the repaired ones

data_with_TVObsAndSaneri = data[data['TVObsKode'].isin([1]) | data['DatoSaneri'] > 0]

In [9]:
#get around 2077 rows with not broken pipes

data_not_broken = data[~data['TVObsKode'].isin([0]) | data['DatoSaneri'] == 0]
data_not_broken = data_not_broken.sample(n=2077) 

In [10]:
frames = [data_with_TVObsAndSaneri, data_not_broken]
  
data_final = pd.concat(frames)
data_final

,XKoordinat,YKoordinat,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
36,529911.05,6252443.83,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0
42,530405.37,6252578.04,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
43,530493.05,6252579.67,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
64,530791.62,6252572.03,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
65,530857.05,6252552.13,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11093,540652.59,6247761.92,3.22,3.942668,29.33,-24.639210,250.0,1.0,1980.0,1,0,0.0,2010,2014,0.0
6133,531206.24,6261720.37,27.88,26.770000,42.81,25.928521,300.0,1.0,1967.0,1,0,0.0,2010,2014,0.0
17354,508979.59,6238763.20,43.88,43.380000,41.12,12.159533,150.0,1.0,1954.0,1,0,0.0,2012,2014,0.0
10046,528358.77,6257976.54,36.68,36.270000,49.77,8.237894,200.0,1.0,1980.0,1,0,0.0,2010,2014,0.0


In [11]:
data = data_final

In [12]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [13]:
#get data copied
datacopy = data


# add  age column

#get current year
from datetime import date
now = date.today().year


def age_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] > 0) :
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])

datacopy['Age'] = datacopy.apply(age_df, axis = 1)

In [14]:
# add a column 'PipeStatus'
# 1 as broken and 0 as not broken

def broken_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] < (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 1
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] >= (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 0
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return 1
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return 0
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return 0

datacopy['PipeStatus'] = datacopy.apply(broken_df, axis = 1)

In [15]:
# datacopy = datacopy.sample(n=22) 
# datacopy

In [16]:
# data_fs= np.where(np.isnan(datacopy))
# data_fs
# row = datacopy.iloc[369] #index=1 => second row
# print(row)

In [17]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(datacopy.shape[0]))
datacopy = datacopy.dropna()
print("Number of rows after removing NaNs: {}".format(datacopy.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [18]:
#drop columns not needed after adding new features

columns_to_be_removed = ['DatoOprett', 'DatoOpdate']
datacopy=datacopy.drop(columns_to_be_removed,axis='columns')
datacopy[0:-1]

,XKoordinat,YKoordinat,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Age,PipeStatus
36,529911.05,6252443.83,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,1997.0,24.0,0
42,530405.37,6252578.04,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
43,530493.05,6252579.67,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
64,530791.62,6252572.03,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
65,530857.05,6252552.13,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21354,512679.65,6246869.79,58.97,57.630000,38.50,34.805195,200.0,1.0,1973.0,1,0,0.0,0.0,48.0,0
11093,540652.59,6247761.92,3.22,3.942668,29.33,-24.639210,250.0,1.0,1980.0,1,0,0.0,0.0,41.0,0
6133,531206.24,6261720.37,27.88,26.770000,42.81,25.928521,300.0,1.0,1967.0,1,0,0.0,0.0,54.0,0
17354,508979.59,6238763.20,43.88,43.380000,41.12,12.159533,150.0,1.0,1954.0,1,0,0.0,0.0,67.0,0


In [19]:
# creating features set and target

columns_to_be_removed = ['PipeStatus']
data_features= datacopy.drop(columns_to_be_removed,axis='columns')
columns_to_be_removed = ['fra_kote','til_kote', 'DiameterIn','MaterialeK','anlag_aar','TransportK','Funktionsk','TVObsKode','DatoSaneri','Age']
data_target=datacopy.drop(columns_to_be_removed,axis='columns')

In [20]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [21]:
#data['XKoordinat'] = data['XKoordinat'].astype(int)

In [22]:
#data['YKoordinat'] = data['YKoordinat'].astype(int)
#data

In [23]:
def get_matched_depth(select_x,select_y):
    #select_x = 526761
    #select_y = data.loc[data['XKoordinat'] == select_x]['YKoordinat'].values[0]
    val = data.loc[data['XKoordinat'] == select_x]['Laengde'].values[0]
    max_x=select_x+val
    min_x=select_x-val

    matched_depth_col = df.loc[(df['XKoordinat'] <= max_x)&(df['XKoordinat'] >= min_x)
    &(df['YKoordinat'] <= select_y+val)&(df['YKoordinat'] >= select_y-val)]['Depth']
    
    #test if there is a value
    if(matched_depth_col.size > 0):
        matched_depth = matched_depth_col.values[0]
    else:
        matched_depth = np.NaN
    
    return matched_depth

In [24]:
def get_matched_depth_1(select_x,select_y):
    #select_x = 526761
    #select_y = data.loc[data['XKoordinat'] == select_x]['YKoordinat'].values[0]
    length = data.loc[data['XKoordinat'] == select_x]['Laengde'].values[0]
    angle = data.loc[data['XKoordinat'] == select_x]['Fald'].values[0]
    #calculate another point by length:
    end_x = select_x+ (length * np.cos(angle))
    end_y = select_y+ (length * np.sin(angle))
    if(end_x > select_x):
        max_x=end_x
        min_x=select_x
    else:
        min_x=end_x
        max_x=select_x   
    if(end_y > select_y):
        max_y=end_y
        min_y=select_y
    else:
        min_y=end_y
        max_y=select_y   


    matched_depth_col = df.loc[(df['XKoordinat'] <= max_x)&(df['XKoordinat'] >= min_x)
    &(df['YKoordinat'] <= max_y)&(df['YKoordinat'] >= min_y)]['Depth']
    #test if there is a value
    if(matched_depth_col.size > 0):
        matched_depth = matched_depth_col.values[0]
    else:
        matched_depth = np.NaN
    
    return matched_depth

In [25]:
select_x = datacopy['XKoordinat']
select_x

36       529911.05
42       530405.37
43       530493.05
64       530791.62
65       530857.05
           ...    
11093    540652.59
6133     531206.24
17354    508979.59
10046    528358.77
20406    514434.10
Name: XKoordinat, Length: 4154, dtype: float64

In [26]:
def add_depth(datacopy):
    select_x = datacopy['XKoordinat']
    select_y = datacopy['YKoordinat']
    return get_matched_depth_1(select_x,select_y)
datacopy['Depth'] = datacopy.apply(add_depth,axis =1)
datacopy    
        

,XKoordinat,YKoordinat,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Age,PipeStatus,Depth
36,529911.05,6252443.83,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,1997.0,24.0,0,NaN
42,530405.37,6252578.04,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,0.0,82.0,1,2.314121
43,530493.05,6252579.67,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,0.0,82.0,1,NaN
64,530791.62,6252572.03,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1,4.478954
65,530857.05,6252552.13,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1,5.462685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11093,540652.59,6247761.92,3.22,3.942668,29.33,-24.639210,250.0,1.0,1980.0,1,0,0.0,0.0,41.0,0,1.429384
6133,531206.24,6261720.37,27.88,26.770000,42.81,25.928521,300.0,1.0,1967.0,1,0,0.0,0.0,54.0,0,8.369598
17354,508979.59,6238763.20,43.88,43.380000,41.12,12.159533,150.0,1.0,1954.0,1,0,0.0,0.0,67.0,0,NaN
10046,528358.77,6257976.54,36.68,36.270000,49.77,8.237894,200.0,1.0,1980.0,1,0,0.0,0.0,41.0,0,NaN


In [27]:
datacopy.dropna()

,XKoordinat,YKoordinat,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Age,PipeStatus,Depth
42,530405.37,6252578.04,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,0.0,82.0,1,2.314121
64,530791.62,6252572.03,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1,4.478954
65,530857.05,6252552.13,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1,5.462685
213,538388.34,6260162.14,52.36,51.470000,105.35,8.448030,250.0,1.0,1975.0,1,0,1.0,0.0,46.0,1,4.192089
261,538790.31,6260380.52,51.87,51.180000,66.66,10.351035,200.0,1.0,1975.0,1,0,1.0,0.0,46.0,1,3.730137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13045,530420.88,6271164.98,8.16,7.280000,44.60,19.730942,200.0,1.0,1967.0,1,0,0.0,0.0,54.0,0,4.507581
18962,530670.19,6274481.60,31.20,31.000000,58.95,3.392706,300.0,1.0,1969.0,1,0,0.0,0.0,52.0,0,2.505411
11093,540652.59,6247761.92,3.22,3.942668,29.33,-24.639210,250.0,1.0,1980.0,1,0,0.0,0.0,41.0,0,1.429384
6133,531206.24,6261720.37,27.88,26.770000,42.81,25.928521,300.0,1.0,1967.0,1,0,0.0,0.0,54.0,0,8.369598


In [28]:
data['Depth'] = data.apply(add_depth,axis =1)
data 

,XKoordinat,YKoordinat,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri,Age,PipeStatus,Depth
36,529911.05,6252443.83,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0,24.0,0,NaN
42,530405.37,6252578.04,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0,82.0,1,2.314121
43,530493.05,6252579.67,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0,82.0,1,NaN
64,530791.62,6252572.03,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0,76.0,1,4.478954
65,530857.05,6252552.13,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0,76.0,1,5.462685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11093,540652.59,6247761.92,3.22,3.942668,29.33,-24.639210,250.0,1.0,1980.0,1,0,0.0,2010,2014,0.0,41.0,0,1.429384
6133,531206.24,6261720.37,27.88,26.770000,42.81,25.928521,300.0,1.0,1967.0,1,0,0.0,2010,2014,0.0,54.0,0,8.369598
17354,508979.59,6238763.20,43.88,43.380000,41.12,12.159533,150.0,1.0,1954.0,1,0,0.0,2012,2014,0.0,67.0,0,NaN
10046,528358.77,6257976.54,36.68,36.270000,49.77,8.237894,200.0,1.0,1980.0,1,0,0.0,2010,2014,0.0,41.0,0,NaN
